**Dataset**
labeled datasset collected from twitter

**Objective**
classify tweets containing hate speech from other tweets.
0 -> no hate speech
1 -> contains hate speech

**Total Estimated Time = 90 Mins**

### Import Libraries

In [23]:
import pandas as pd
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer  
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model

import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from collections import Counter
from pathlib import Path
import os
import numpy as np
import re
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import wordnet
import unicodedata
import html

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Load Dataset

In [3]:
DF = pd.read_csv("dataset.csv", index_col='id')
DF

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation
...,...,...
31958,0,ate @user isz that youuu?ðððððð...
31959,0,to see nina turner on the airwaves trying to...
31960,0,listening to sad songs on a monday morning otw...


### EDA

- check NaNs

In [4]:
DF.isnull().sum()

label    0
tweet    0
dtype: int64

- check duplicates

In [5]:
DF.duplicated().sum()

2432

In [6]:
DF_new = DF.drop_duplicates()
DF_new

,label,tweet
id,,
1,0,@user when a father is dysfunctional and is s...
2,0,@user @user thanks for #lyft credit i can't us...
3,0,bihday your majesty
4,0,#model i love u take with u all the time in ...
5,0,factsguide: society now #motivation
...,...,...
31957,0,off fishing tomorrow @user carnt wait first ti...
31958,0,ate @user isz that youuu?ðððððð...
31959,0,to see nina turner on the airwaves trying to...


- show samples of data texts to find out required preprocessing steps

In [7]:
with pd.option_context('display.max_colwidth', None):
  display(DF_new.sample(10))

,label,tweet
id,,
19178,0,only smiles ðððªðªððð #smile #goodsunday day #nighttime #instalike #instafollowâ¦
9231,0,"i want my boyfriend &amp; chinese food, but my boyfriends @ work &amp; i'm supposed to be on a diet."
31242,0,@user #djhumility feeling you with the ojb vibes... #ripojb #areacodemixshow cc @user @user @user
18416,0,"@user learn, grow, soar! #confidence #positivevibes #success #lovemylife #tenpluslife"
2796,0,"@user @user @user of your last #topten stories, 8 were on medical workers strike! the state of healthcaâ¦"
30136,0,...music #jazz #music #jazzmusic #jazzchillout tune #guitarjazz #pianojazz #inspire friday
27176,0,2 ways to b in #life- never take help of tears to show ur emotions..&amp; never take help of words to show ur anger..
17308,0,i am wondrous. #i_am #positive #affirmation
4712,0,@user people actually believe the shit spewing out of @user mouth...


- check dataset balancing

In [8]:
print("Number of samples with label 0 = ", DF_new[DF_new.label == 0].shape[0])
print("Number of samples with label 1 = ", DF_new[DF_new.label == 1].shape[0])

Number of samples with label 0 =  27517
Number of samples with label 1 =  2013


- Cleaning and Preprocessing are:
    - 1
    - 2
    - 3

### Cleaning and Preprocessing

In [9]:
def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)

def remove_non_ascii(text):
    """Remove non-ASCII characters from list of tokenized words"""
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

def to_lowercase(text):
    return text.lower()

def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)

def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]

def stem_words(words):
    """Stem words in text"""
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in words]

def text2words(text):
  return word_tokenize(text)

def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))

def RemoveURLs(words):
  return [t for t in words if t.startswith('http')==False]


stop_words = stopwords.words('english')

def normalize_text( text):
    text = remove_special_chars(text)
    text = remove_non_ascii(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    words = stem_words(words)# Either stem ovocar lemmatize
    words = RemoveURLs(words)
    # words = lemmatize_words(words)
    # words = lemmatize_verbs(words)

    return words

In [10]:
NewColList = []
for i in DF_new.index:
  NewColList.append(  " ".join (normalize_text(DF_new.loc[i, 'tweet'])))

In [11]:
DF_new['NormalizedTweets'] = NewColList
with pd.option_context('display.max_colwidth', None):
  display(DF_new.sample(10))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,label,tweet,NormalizedTweets
id,,,
25698,0,@user chicago is a nice and windy city ð¨ð¨ #beautiful #picture #yummy #holiday,user chicago nice windi citi beauti pictur yummi holiday
2979,0,"some chick just posted a ""hot"" pic of herself on a lounge chair. her caption/reason: ""cooking on the grill. happy father's day.""",chick post hot pic loung chair captionreason cook grill happi father day
17506,0,my new song #summermood,new song summermood
17503,0,#queengreen and her cub #simba celebrating the never ending circle of life #queens90thbihday #queenat90,queengreen cub simba celebr never end circl life queensthbihday queenat
26261,1,naked woman mo fo sex,nake woman mo fo sex
24188,0,"making my list today. saturday y'all! #advicebypg ð #pgpad #nashville xo, pg",make list today saturday yall advicebypg pgpad nashvil xo pg
24678,0,who has two thumbs and a new job? this girl!,two thumb new job girl
16009,0,a grand a week in benefits? a grand a week? stop having kids you absolute lazy pieces of scum!! #onbenefits #whybother?,grand week benefit grand week stop kid absolut lazi piec scum onbenefit whyboth
28144,0,happy father's day! #xyzprinting #xyz #3d #printing #printer #technology #becreative #maker #fathersday ...,happi father day xyzprint xyz print printer technolog becr maker fathersday


In [12]:
x_train, x_val, y_train, y_val = train_test_split(DF_new['NormalizedTweets'], DF_new['label'], test_size=0.2, random_state=42)

**If it takes 60 Mins till here, you are doing Great** <br>
**If not! You also are doing Great**

### Modelling

Count Vectorizer + Logistic Regression

In [27]:
vec = CountVectorizer()
clf = LogisticRegression()
make_pipeline(vec, clf)
pipe = make_pipeline(vec, clf)
pipe.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


#### Evaluation

In [37]:
from sklearn import metrics

def print_report(pipe, x_val, y_val):
    y_pred = pipe.predict(x_val)
    report = metrics.classification_report(y_val, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_val, y_pred)))

print_report(pipe, x_val, y_val)

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      5498
           1       0.84      0.43      0.57       408

    accuracy                           0.96      5906
   macro avg       0.90      0.71      0.77      5906
weighted avg       0.95      0.96      0.95      5906

accuracy: 0.955


### Enhancement

- Using different N-grams
- Using different text representation technique

## Tfidf with N-grams


In [47]:
vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4))
clf = LinearSVC()
pipe_tfidf = make_pipeline(vec, clf)
pipe_tfidf.fit(x_train, y_train)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4))),
                ('linearsvc', LinearSVC())])

In [48]:
print_report(pipe_tfidf, x_val, y_val)

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      5498
           1       0.84      0.56      0.67       408

    accuracy                           0.96      5906
   macro avg       0.91      0.77      0.83      5906
weighted avg       0.96      0.96      0.96      5906

accuracy: 0.962


#### Done!